# مقایسه کرونا  (کوید- ۱۹) در ایران با باقی کشورها
>مقایسه تغییرات افزایش تعداد مبتلایان کرونا در ایران با ایتالیا ، کره جنوبی ، آمریکا و ژاپن 

- comments: true
- author: پراتاب واردهان 
- categories: [افزایش, مقایسه]
- image: images/covid-compare-country-trajectories.png
- permalink: /compare-country-trajectories/

In [1]:
#hide
!pip install jdatetime
!pip install persiantools
import pandas as pd
import altair as alt
import jdatetime
from persiantools import digits
from IPython.display import HTML

You should consider upgrading via the '/opt/hostedtoolcache/Python/3.6.11/x64/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/hostedtoolcache/Python/3.6.11/x64/bin/python -m pip install --upgrade pip' command.


In [2]:
#hide
url = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df = pd.read_csv(url)
# rename countries
df['Country/Region'] = df['Country/Region'].replace({'Korea, South': 'South Korea'})
# Add countries
countries = ["China", "Italy", "Spain", "France", "United Kingdom", "Germany", 
             "US", "South Korea", "Japan", 
             "Iran", "India", "Canada", "Australia"]
df = df[df['Country/Region'].isin(countries)]   # Just use list above
df['Country/Region'] = df['Country/Region'].replace({'Italy': 'ایتالیا'})
df['Country/Region'] = df['Country/Region'].replace({'China': 'چین'})
df['Country/Region'] = df['Country/Region'].replace({'Spain': 'اسپانیا'})
df['Country/Region'] = df['Country/Region'].replace({'France': 'فرانسه'})
df['Country/Region'] = df['Country/Region'].replace({'Germany': 'آلمان'})
df['Country/Region'] = df['Country/Region'].replace({'United Kingdom': 'انگلیس'})
df['Country/Region'] = df['Country/Region'].replace({'US': 'آمریکا'})
df['Country/Region'] = df['Country/Region'].replace({'South Korea': 'کره جنوبی'})
df['Country/Region'] = df['Country/Region'].replace({'Japan': 'ژاپن'})
df['Country/Region'] = df['Country/Region'].replace({'Iran': 'ایران'})
df['Country/Region'] = df['Country/Region'].replace({'India': 'هند'})
df['Country/Region'] = df['Country/Region'].replace({'Canada': 'کانادا'})
df['Country/Region'] = df['Country/Region'].replace({'Australia': 'استرالیا'})
dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]

In [3]:
#hide
dff = (df.groupby('Country/Region')[dt_cols].sum()
       .stack().reset_index(name='Confirmed Cases')
       .rename(columns={'level_1': 'Date', 'Country/Region': 'Country'}))
dff['Date'] = pd.to_datetime(dff['Date'], format='%m/%d/%y')

In [4]:
#hide
MIN_CASES = 700
LAST_DATE = dt_cols[-1]
# sometimes last column may be empty, then go backwards
for c in dt_cols[::-1]:
    if not df[c].fillna(0).eq(0).all():
        LAST_DATE = c
        break
countries = dff[dff['Date'].eq(LAST_DATE) & dff['Confirmed Cases'].ge(MIN_CASES) & 
        dff['Country'].ne('چین')
       ].sort_values(by='Confirmed Cases', ascending=False)
countries = countries['Country'].values

In [5]:
#hide
SINCE_CASES_NUM = 100
dff2 = dff[dff['Country'].isin(countries)].copy()
days_since = (dff2.assign(F=dff2['Confirmed Cases'].ge(SINCE_CASES_NUM))
              .set_index('Date')
              .groupby('Country')['F'].transform('idxmax'))
dff2['Days since 100 cases'] = (dff2['Date'] - days_since.values).dt.days.values
dff2 = dff2[dff2['Days since 100 cases'].ge(0)]

In [6]:
#hide
def get_country_colors(x):
    mapping = {
        'ایران': 'black',
        'ایتالیا': '#A569BD',
        'چین': '#A1BA59',
        'اسپانیا': '#D5F5E3',
        'آمریکا': '#FF5733', 
        'کره جنوبی': '#A9DFBF',
        'ژاپن': '#A9CCE3',
        'فرانسه': '#EB984E',
        'آلمان': '#E59866',
        'انگلیس': '#D7BDE2',
        'هند': '#C1B7AD',
        'کانادا': '#F9E79F',
        'استرالیا': '#ABEBC6'}
    return mapping.get(x, '#C1B7AD')

In [7]:
#hide_input
baseline_countries = ['آمریکا','ایتالیا', 'کره جنوبی', 'ژاپن']
max_date = dff2['Date'].max()
color_domain = list(dff2['Country'].unique())
color_range = list(map(get_country_colors, color_domain))

def make_since_chart(highlight_countries=[], baseline_countries=baseline_countries):
    selection = alt.selection_multi(fields=['Country'], bind='legend', 
                                    init=[{'Country': x} for x in highlight_countries + baseline_countries])
    
    base = alt.Chart(dff2, width=550).encode(
        x=alt.X('Days since 100 cases:Q', axis=alt.Axis(titleFont='Tahoma', title=' تعداد روز بعد از ۱۰۰ مورد اول')),
        y=alt.Y('Confirmed Cases:Q', scale=alt.Scale(type='log'), axis=alt.Axis(titleFont='Tahoma', title='موارد ثبت شده')),
        color=alt.Color(
            'Country:N',
        scale=alt.Scale(domain=color_domain, range=color_range),
        legend=alt.Legend(columns=5, orient='bottom',title='کشور',labelFont='Tahoma',labelFontSize=13,titleFontSize=14, titleFont='Tahoma', symbolLimit=len(color_domain))),
        tooltip=[alt.Tooltip('Country:O', title='کشور'),alt.Tooltip('Date:O', title='تاریخ'),alt.Tooltip('Confirmed Cases:O', title='موارد ثبت شده'),alt.Tooltip('Days since 100 cases:O', title=' تعداد روز بعد از ۱۰۰ مورد اول')],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    )
    max_day = dff2['Days since 100 cases'].max()
    ref = pd.DataFrame([[x, 100*1.33**x] for x in range(max_day+1)], columns=['Days since 100 cases', ' Confirmed Cases'])
    base_ref = alt.Chart(ref).encode(x='Days since 100 cases:Q', y='Confirmed Cases:Q')
    return (
        base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]) +
        base_ref.transform_filter(
            alt.datum['Days since 100 cases'] >= max_day
        ).mark_text(dy=-6, align='right', fontSize=14, color='#FF5733', text='۳۳٪ رشد روزانه') +
        base.mark_line(point=True).add_selection(selection) + 
        base.transform_filter(
            alt.datum['Date'] >= int(max_date.timestamp() * 1000)
        ).mark_text(dy=-8, align='right', fontWeight='bold').encode(text='Country:N')
    ).properties(
        title=f"مقایسه رشد تعداد کل مبتلایان در ایران  با شرایط  {', '.join(baseline_countries)}"
    ).configure_title(fontSize=15, font='Tahoma', color='#24292e'
    ).configure_axis(
    grid=False
    ).configure_view(
    strokeOpacity=0
    )


## یادگیری از ایتالیا ، کره جنوبی ، آمریکا و ژاپن

**ایتالیا** ، **کره جنوبی**، **ژاپن** و **آمریکا** چهار کشوری هستند که نرخ رشد متفاوتی را نشان داده اند. برای مثال کره جنوبی بعد از گذشت دو هفته از شناسایی ۱۰۰ مورد اولش ، شاهد کاهش رشد است. **ایتالیا** و **آمریکا** بعد از هفته سوم همچنان شاهد رشد در تعداد مبتلایان میباشند. **ایران** رشد کمتری نسبت به آمریکا و ایتالیا نشان میدهد ولی نسبت به کره جنوبی و ژاپن شاهد رشد بیشتری نشان داده است. 

میتوانید در نمودار زیر مشاهده کنید.

برای مشاهده چند مورد باهم دکمه شیفت را فشار دهید 


In [8]:
#hide_input
gredate = pd.to_datetime(LAST_DATE)
jalili_date = jdatetime.date.fromgregorian(year=gredate.year,month=gredate.month,day=gredate.day,locale='fa_IR').strftime("%d / %m / %Y")
jalili_date = digits.en_to_fa(jalili_date)
HTML(f'<small class="float-right jdate"> آخرین بروزرسانی در :   {jalili_date} </small>')

In [9]:
#hide_input
chart2 = make_since_chart(['ایران'])
chart2

alt.LayerChart(...)

کشور مورد نظر خود را انتخاب کنید:

In [10]:
#hide_input
base = alt.Chart(dff2, width=600).encode(
    x=alt.X('Days since 100 cases:Q', axis=alt.Axis(titleFont='Tahoma', title=' تعداد روز بعد از ۱۰۰ مورد اول')),
    y=alt.Y('Confirmed Cases:Q', scale=alt.Scale(type='log'), axis=alt.Axis(titleFont='Tahoma', title='موارد ثبت شده')),
    color=alt.Color('Country:N', scale=alt.Scale(domain=color_domain, range=color_range), 
                   legend=alt.Legend(columns=5, orient='bottom',title='کشور',labelFont='Tahoma',labelFontSize=13,titleFontSize=14, titleFont='Tahoma', symbolLimit=len(color_domain))),
    tooltip=[alt.Tooltip('Country:O', title='کشور'),alt.Tooltip('Date:O', title='تاریخ'),alt.Tooltip('Confirmed Cases:O', title='موارد ثبت شده'),alt.Tooltip('Days since 100 cases:O', title=' تعداد روز بعد از ۱۰۰ مورد اول')]
)

country_selection = alt.selection_single(
    name='Select', fields=['Country'], 
    bind=alt.binding_select(options=list(sorted(set(countries) - set(baseline_countries)))),
    init={'Country': 'ایران'})

date_filter = alt.datum['Date'] >= int(max_date.timestamp() * 1000)
base2 = base.transform_filter(alt.FieldOneOfPredicate('Country', baseline_countries))
base3 = base.transform_filter(country_selection)
base4 = base3.transform_filter(date_filter)

max_day = dff2['Days since 100 cases'].max()
ref = pd.DataFrame([[x, 100*1.33**x] for x in range(max_day+1)], columns=['Days since 100 cases', 'Confirmed Cases'])
base_ref = alt.Chart(ref).encode(x='Days since 100 cases:Q', y='Confirmed Cases:Q')
base_ref_f = base_ref.transform_filter(alt.datum['Days since 100 cases'] >= max_day)

chart5 = (
 base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]) + 
 base_ref_f.mark_text(dy=-6, align='right', fontSize=14, color='#FF5733', text='۳۳٪ رشد روزانه') + 
 base2.mark_line(point=True, tooltip=True) +
 base3.mark_line(point={'size':50}, tooltip=True) +
 base2.transform_filter(date_filter).mark_text(dy=-8, align='right').encode(text='Country:N') +
 base4.mark_text(dx=8, align='left', fontWeight='bold').encode(text='Country:N') +
 base4.mark_text(dx=8, dy=12, align='left', fontWeight='bold').encode(text='Confirmed Cases:Q')
).add_selection(country_selection).properties(
    title=f"مقایسه با کشورهای {', '.join(baseline_countries)}"
).configure_title(fontSize=15, font='Tahoma', color='#24292e').configure_axis(
    grid=False
    ).configure_view(
    strokeOpacity=0
    )
chart5

alt.LayerChart(...)


تحلیل اصلی از [پراتاپ واردهان](https://twitter.com/PratapVardhan)[^۱]

[^۱]: منبع: ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). لینک به صفحه [اصلی](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-compare-country-trajectories.ipynb).